In [1]:
#Dependencies
import os
import numpy as np
import random
import pandas as pd
import torch
import copy
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.nn import Linear, ReLU, LeakyReLU, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torchvision import datasets, transforms
from math import sqrt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from torch.autograd import Variable
import torch.optim as optim
import torch.autograd as autograd
import copy
# from tensorflow import keras
import thread6 as thread
device = torch.device("cuda")
from torch.optim.lr_scheduler import StepLR,  ExponentialLR


In [2]:
#training operations
def downsampling_operator(s):
    S = np.zeros((np.int_(64/s), 64))
    vector = np.zeros((1, 64))
    vector[0,0] = 1

    for i in range(np.int_(64/s)):
        S[i,:] = vector
        vector = np.roll(vector, s, axis=1)
    return torch.from_numpy(S).float()


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def add_noise_to_training_data(training_data_pred_ground_truth, noise_sensor=0.03):
        training_data_pred_noisy = np.zeros(training_data_pred_ground_truth.shape)
        for i in range(training_data_pred_ground_truth.shape[0]):
            gt = training_data_pred_ground_truth[i,:,:,:]
            noise = np.random.normal(0, sensor_noise, gt.shape) # mu, sigma, size
            noise[gt == 0] = 0
            training_data_pred_noisy[i,:,:,:] = gt + noise
        return training_data_pred_noisy

def pretraining(DnCnn_model, num_epochs, learning_rate, dataloader_pretrain, device, l2_w, 
                 batch_size):
    #  criterion = nn.MSELoss()
     criterion = nn.L1Loss()
     optimizer = torch.optim.Adam(DnCnn_model.parameters(), lr=learning_rate, weight_decay=l2_w)
 
     for epoch in range(num_epochs):
        acc_loss = 0.
        for data_noisy, clean in dataloader_pretrain:
            if batch_size>1:
                batch_size = len(data_noisy)           
            data = (data_noisy.float()).to(device)
            clean = (clean.float()).to(device) 
            data = torch.moveaxis(data, 3, 2)
            data = torch.moveaxis(data, 2, 1)

            clean = torch.moveaxis(clean, 3, 2)
            clean = torch.moveaxis(clean, 2, 1)
            # ===================forward=====================
            output = DnCnn_model(data)       
            loss = torch.sqrt(criterion(output, clean))
            #loss = (criterion(output, clean))
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            acc_loss += loss.item()
        print(acc_loss / len(dataloader_pretrain))
        
def unrolling_model_training(my_sc_model, num_epochs, learning_rate, dataloader_train, 
                               device, l2_w, batch_size, 
                               S, blocks):
    my_sc_model.train()
    # criterion = nn.MSELoss()
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(my_sc_model.parameters(), lr=learning_rate)

    # optimizer = torch.optim.SGD(my_sc_model.parameters(), lr=learning_rate)    
    for epoch in range(num_epochs):
       acc_loss = 0.
       for data, clean in dataloader_train:
           
           batch_size = len(data)           
           data = (data.float()).to(device)
           clean = (clean.float()).to(device)
           
           data = torch.moveaxis(data, 3, 2)
           data = torch.moveaxis(data, 2, 1)
           clean = torch.moveaxis(clean, 3, 2)
           clean = torch.moveaxis(clean, 2, 1)

           # ===================forward=====================

           output = my_sc_model(data, S, batch_size, device, blocks)  
           loss = criterion(output, clean)
           # ===================backward====================
           optimizer.zero_grad()
           loss.backward()
           optimizer.step()
           acc_loss += loss.item()
       #if epoch%10 == 0:
         #torch.save(my_sc_model,"/content/drive/MyDrive/Colab Notebooks/DEEP_UNROLLING_phd/models/fast_eq_50_50x50.pt")

       print(acc_loss / len(dataloader_train))

In [3]:
#define models architectures
def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()


class DnCnn_net(nn.Module):
       def __init__(self, num_of_layers, features, input_channels):
         super().__init__()
         kernel_size = 3
         padding = 1
         features = features
         channels = input_channels
         num_of_layers = num_of_layers
         layers = []
         layers.append((nn.Conv2d(in_channels=channels, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False)))
         layers.append(nn.ReLU())  
         layers.append(nn.Dropout(0.05))

        #  layers.append(nn.GroupNorm(8, 64))
        # nn.utils.spectral_norm
      
         for _ in range(num_of_layers-2):
             layers.append((nn.Conv2d(in_channels=features, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False)))

             layers.append(nn.ReLU())   
             layers.append(nn.Dropout(0.05))
            #  layers.append(nn.GroupNorm(8, features))
         layers.append((nn.Conv2d(in_channels=features, out_channels=channels, kernel_size=kernel_size, padding=padding, bias=False)))
         self.dncnn = (nn.Sequential(*layers))
        
       def forward(self, input):
           return self.dncnn(input) 
       
class deep_unrolled_sc(Module):   
    def __init__(self, DnCnn):
        super(deep_unrolled_sc, self).__init__()
        self.DnCnn = DnCnn
        self.alpha=nn.Parameter(torch.tensor(5.99), requires_grad=True)

    # Defining the forward pass    
    def forward(self, data, S, batch_size, device, blocks):
        # low_dim = S.shape[0]
        high_dim = S.shape[1]        
        
        S = (S.repeat(batch_size, 1, 1))
        ST = torch.transpose(S.float(),1,2)
        STS = torch.matmul(ST ,S)
        STS = torch.unsqueeze(STS, dim=1).to(device)
        
        II = torch.eye(S.shape[2])
        II = (II.repeat(batch_size, 1, 1))
        II = torch.unsqueeze(II, dim=1).to(device)

        A = torch.matmul(torch.unsqueeze(ST.to(device), dim=1), data)
        A = self.alpha*A
        DtD = torch.linalg.inv(II + STS*self.alpha)

        #1
        Z = (torch.ones(batch_size, 1 , high_dim, data.shape[3])).to(device)
        
        
        for kk in range(blocks):
            x = torch.matmul(DtD, (A+ Z))
            Z = self.DnCnn(x)
        return Z




In [4]:
#Federated Learning Functions

def clinet_update_unrolling_model(model, num_epochs, learning_rate, dataloader_train, 
                               device, l2_w, batch_size, 
                               S, blocks, optimizer):
    model.train()
    criterion = nn.L1Loss()

    # optimizer = torch.optim.SGD(my_sc_model.parameters(), lr=learning_rate)    
    for epoch in range(num_epochs):
    #    acc_loss = 0.
       for data, clean in dataloader_train:
           
           batch_size = len(data)           
           data = (data.float()).to(device)
           clean = (clean.float()).to(device)
           
           data = torch.moveaxis(data, 3, 2)
           data = torch.moveaxis(data, 2, 1)
           clean = torch.moveaxis(clean, 3, 2)
           clean = torch.moveaxis(clean, 2, 1)

           # ===================forward=====================
           output = model(data, S, batch_size, device, blocks)   
           loss = criterion(output, clean)
           
           # ===================backward====================
           optimizer.zero_grad()
           loss.backward()
           optimizer.step()
        #    acc_loss += loss.item()

def server_aggregate_fedavg(global_model, client_models):
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = (torch.stack([client_models[i].state_dict()[k].float() for i in range(len(client_models))], 0).mean(0))
    global_model.load_state_dict(global_dict)
    for model in client_models:
        model.load_state_dict(global_model.state_dict())


def ouster_data_evaluation(global_model, test_data_input, test_data, device, S, blocks):
        l1_error = []
        predicted_range_images = np.zeros_like(test_data)

        # my_sc_model.eval()
        # enable_dropout(my_sc_model)

        for i in range(0, test_data.shape[0]):
                # print('Test image: ', i)
                low_res = test_data_input[i,:,:,:]
                high_res = test_data[i,:,:,:]

                low_res = torch.from_numpy(low_res).to(device)
                high_res = torch.from_numpy(high_res).to(device)
                low_res = torch.unsqueeze(low_res, dim=0)
                high_res = torch.unsqueeze(high_res, dim=0)

                low_res = torch.moveaxis(low_res, 3, 2)
                low_res = torch.moveaxis(low_res, 2, 1)

                high_res = torch.moveaxis(high_res, 3, 2)
                high_res = torch.moveaxis(high_res, 2, 1)

                output = global_model(low_res.float(), S, 1, device, blocks)  
                
                output[output < 0] = 0
                output[output > 1] = 1
                
                high = output[0,0:,:,:].cpu().detach().numpy()
                high = np.squeeze(high, axis=0)
                predicted_range_images[i,:,:,0] = high

                criterion = nn.L1Loss()
                l1_error.append(criterion(output, high_res).item())

        return np.mean(l1_error)



In [5]:
#read The training and Testing Data
class CustomDataDataset(Dataset):
    def __init__(self, data, labels):
        self.labels = (labels)
        self.data = (data)

    def __len__(self):
            return len(self.labels)
    
    def __getitem__(self, idx):
            label = self.labels[idx]
            data = self.data[idx,:]
            sample = {"Data": data, "Class": label}
            return data, label


from data_paper import *
training_data_input, training_data_pred_ground_truth = load_train_data()

test_data_input, test_data = load_test_data()


ouster_range_image
##################################################
Input image resolution:   16 by 1024
Output image resolution:  64 by 1024
Upscaling ratio:          4
##################################################
Sensor minimum range:     2.0
Sensor maximum range:     80.0
Sensor view angle:        -16.6 to 16.6
Range normalize ratio:    100.0
##################################################
Training data-set:        /home/alexgi/Workspace/lidar_super_resolution/Lidar Super-Resolution/carla_ouster_range_image.npy 
Testing data-set:         /home/alexgi/Workspace/lidar_super_resolution/Lidar Super-Resolution/ouster_range_image.npy
add noise level: 0.03
1


In [6]:
#@title Homo Encryption FL functions

import time
import tenseal as ts
import numpy as np

def pretty_print_row(row):
  return ' '.join('%03s' % i for i in row)

# write the data to the output files
def write_data(reals, decrs):
  with open('real.txt', 'w') as f:
    for row in reals:
      f.write('[%s]' % pretty_print_row(row) + '\n')

  with open('decr.txt', 'w') as f:
    for row in decrs:
      f.write('[%s]' % pretty_print_row(row) + '\n')

  with open('precision.txt', 'w') as f:
    for i in range(len(reals)):
      prec = []
      for a, b in zip(reals[i], decrs[i]):
        x = abs(a-b)
        prec.append(f'%.{str(x)[-1]}f' % x)
      # prec = np.array(prec, dtype='object')
      f.write(str(prec).replace("'", "")+'\n')

# unused - left for HE demonstration
def homomorphic(parameters):
  for i in range(0, len(parameters)-4, 2):
    v1 = parameters[i].detach().numpy()
    v2 = parameters[i+2].detach().numpy()

    enc_v1 = ts.ckks_vector(context, v1.flat)
    enc_v2 = ts.ckks_vector(context, v2.flat)

    print('P1 + P2 =\n', (v1+v2)[:10])

    encv1v2 = enc_v1 + enc_v2

    dec = np.array(encv1v2.decrypt())
    print('E(P1) + E(P2) = E(P1 + P2) =\n', dec[:10])
    print('-'*20)

#Function to convert model to numpy array
def model_to_matrix(model, num_of_layers, num_of_feat, num_of_input_channels, max_col_dim):
    global_dict = model.state_dict()
    num_of_filters = (num_of_layers-1)*num_of_feat + num_of_input_channels 
    list_of_filters = []
    for cnt, k in enumerate(global_dict.keys()):
        # print(k)
        a = (model.state_dict()[k]) 
        # print(a.shape[0])       
        if k == 'alpha':
            vector = torch.zeros((1, max_col_dim))
            vector[:,0:1] = copy.deepcopy(a)
        else: 
            vector = torch.zeros((a.shape[0], max_col_dim))
            vector[:,0:a.shape[1]*a.shape[2]*a.shape[3]] = copy.deepcopy(torch.reshape(a, (a.shape[0], a.shape[1]*a.shape[2]*a.shape[3])))
        if cnt ==0:
            m_model = vector
        else:
            m_model = torch.cat((m_model, vector), 0)
        if k== 'alpha':  
            list_of_filters.append(1)
        else:
            list_of_filters.append(a.shape[0])
    return m_model, list_of_filters

def from_matrix_to_model(mavg, num_of_layers, num_of_feat, num_of_input_channels, list_of_filters, num_clients):
    model_avg = (copy.deepcopy(global_model)).to(device)
    global_dict = model_avg.state_dict()
    start_row = 1
    for cnt, k in enumerate(global_dict.keys()):
        a = (model_avg.state_dict()[k])         
        if k=='alpha':
            select = mavg[cnt]
            select = torch.from_numpy(select)
        else:
            end_row= start_row + list_of_filters[cnt]
            select = mavg[start_row:end_row]
            select  = np.vstack((np.hstack(a) for a in select))
            select = torch.reshape(torch.from_numpy(select), (a.shape[0], a.shape[1], a.shape[2], a.shape[3]))
            start_row = end_row              

        global_dict[k] = ((select)/num_clients).float()
        
    model_avg.load_state_dict(global_dict)  
    return model_avg


def homomorphic_encryption_process_A(matrix_models, num_clients):
    
    # setup TenSEAL context
    context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=4096)
    context.global_scale = 2**40
    
    reals = []
    decrs = []
    


    start = int(time.time())
    for i in range(matrix_models[0].shape[0]):
        row0 = matrix_models[0][i]
        row1 = matrix_models[1][i]
        row2 = matrix_models[2][i]
        row3 = matrix_models[3][i]
        row4 = matrix_models[4][i]    
        # row5 = matrix_models[5][i]
        # row6 = matrix_models[6][i]
        # row7 = matrix_models[7][i]
        # row8 = matrix_models[8][i]
        # row9 = matrix_models[9][i]

        # remove null values
        row0 = row0[row0 != 0.0]
        row1 = row1[row1 != 0.0]
        row2 = row2[row2 != 0.0]
        row3 = row3[row3 != 0.0]
        row4 = row4[row4 != 0.0]
        # row5 = row5[row5 != 0.0]
        # row6 = row6[row6 != 0.0]
        # row7 = row7[row7 != 0.0]
        # row8 = row8[row8 != 0.0]
        # row9 = row9[row9 != 0.0]

        # encrypt each row
        enc1 = ts.ckks_vector(context, row0)
        enc2 = ts.ckks_vector(context, row1)
        enc3 = ts.ckks_vector(context, row2)
        enc4 = ts.ckks_vector(context, row3)
        enc5 = ts.ckks_vector(context, row4)
        # enc6 = ts.ckks_vector(context, row5)
        # enc7 = ts.ckks_vector(context, row6)
        # enc8 = ts.ckks_vector(context, row7)
        # enc9 = ts.ckks_vector(context, row8)
        # enc10 = ts.ckks_vector(context, row9)    
    
        # add the encrypted rows
        enct = enc1+enc2+enc3+enc4+enc5
    
        # decrypt the encrypted row sum
        decr = np.array(enct.decrypt()).flatten()
        decrs.append(decr)
    decrs = np.array(decrs, dtype='object')
    end = int(time.time())
    
    return decrs

def homomorphic_encryption_process_B(matrix_models, num_clients):
    # setup TenSEAL context
    context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=4096)
    context.global_scale = 2**40
    decrs = []
    start = int(time.time())
    for i in range(matrix_models[0].shape[0]):
        encrypted_rows = []
        for j in range(num_clients):
            row = matrix_models[j][i]
            row = row[row != 0.0]
            enc_row = ts.ckks_vector(context, row)
            encrypted_rows.append(enc_row)
        # add the encrypted rows
        encrypted_total = encrypted_rows[0]
        for enc_row in encrypted_rows[1:]:
            encrypted_total += enc_row
        # decrypt the encrypted row sum
        decr = np.array(encrypted_total.decrypt()).flatten()
        decrs.append(decr)

    decrs = np.array(decrs, dtype='object')
    end = int(time.time())

    return decrs

In [7]:
#Hyperparameters
num_clients = 4
batch_size = 6

#Splitting the data
traindata = CustomDataDataset(training_data_input, training_data_pred_ground_truth)
traindata_split = torch.utils.data.random_split(traindata, [int(traindata.data.shape[0] / num_clients) for _ in range(num_clients)])
dataloader_train = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]

S = downsampling_operator(4)

In [8]:
# Instantiate models and optimizers

num_of_layers = 6
blocks = num_of_layers
num_of_feat = 64
num_of_input_channels = 1
max_col_dim =  num_of_feat *3*3

# Instantiate models and optimizers
global_model = deep_unrolled_sc(DnCnn_net(num_of_layers, 64, 1).to(device)).to(device)
client_models = [copy.deepcopy(global_model) for _ in range(num_clients)]

In [ ]:
#Run Federated Learning Rounds

use_homo_encryption = True #Select if you want to choose homo-encryption

num_rounds = 60
epochs = 5
learning_rate = 1e-3
l2_w = 1e-7
## Define the end learning rate
end_lr = 1e-4
# Define the decay factor
decay_factor = (end_lr / learning_rate) ** (1 / num_rounds)
optimizer_init = torch.optim.Adam(copy.deepcopy(global_model).parameters(), lr = learning_rate)
# scheduler = ExponentialLR(optimizer_init, gamma=decay_factor)

model_dir = os.path.join('/home/alexgi/Workspace/lidar_super_resolution/Lidar Super-Resolution/Federated Learning/FL_DU_models',
                        f'use_homo_encryption_6blocks_dropout005{use_homo_encryption}')
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
loss_no = []
for r in range(num_rounds):
    #client update
    opt = [torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_w) for model in client_models]
 
    for i in range(num_clients):
        print('Client: ', i)
        clinet_update_unrolling_model(client_models[i], epochs, learning_rate, dataloader_train[i], 
                               device, l2_w, batch_size, 
                               S, blocks, opt[i])
    
    
    if use_homo_encryption == True:
        #encrypt models and perform aggregation in the encrypted domain ----------------------------------------------------------------------------
        matrix_models = []
        # from model structure to a matrix
        for i in range(num_clients):
            m, list_of_filters = model_to_matrix(client_models[i], num_of_layers, num_of_feat, num_of_input_channels, max_col_dim)    
            matrix_models.append(copy.deepcopy(m))    
        # encrypt. fuse the enrypted models and decrypt the result
        decrypted_global_model = homomorphic_encryption_process_B(matrix_models, num_clients)
        #from the matrix back to model structure
        global_model = from_matrix_to_model((decrypted_global_model), num_of_layers, num_of_feat, num_of_input_channels, list_of_filters, num_clients)
        # Load the global model to the clients models
        for model in client_models:
            model.load_state_dict(global_model.state_dict())
    else:
        ### Perform simple server aggregate 
        server_aggregate_fedavg(global_model, client_models)
        
    # Save the entire global model at the end of each round
    model_path = os.path.join(model_dir, f'global_model_round_{r}.pt')
    torch.save(global_model, model_path)
    
    scheduler.step()
    learning_rate = optimizer_init.param_groups[0]['lr']
    
    
    # #Evaluate global model
    l1_error = ouster_data_evaluation(global_model, test_data_input[0:2000,:], test_data[0:2000,:], device, S, blocks)
    loss_no.append(l1_error)
    
    print('Round: ', r, 'L1 error: ', l1_error)
    
    
array_path = os.path.join(model_dir, 'loss.npy')
np.save(array_path, loss_no)
# print(min(loss))  

In [ ]:
    # Save the entire global model at the end of each round
    model_path = os.path.join(model_dir, f'global_model_round_{r}.pt')
    torch.save(global_model.state_dict(), model_path)

In [ ]:
# #Evaluate global model
l1_error = ouster_data_evaluation(global_model, test_data_input[0:6000,:], test_data[0:6000,:], device, S, blocks)
# loss.append(l1_error)

In [ ]:
print(l1_error)

In [12]:
def MC_drop(iterate_count=10):

    test_data_input, _ = load_test_data()
    # test_data_input = np.load('zero_dropoff_data.npy')

    this_test = np.empty([iterate_count, image_rows_low, image_cols, channel_num], dtype=np.float32)
    test_data_prediction = np.empty([test_data_input.shape[0], image_rows_high, image_cols, 2], dtype=np.float32)
    
    # global_model.eval()
    enable_dropout(global_model)


    # for i in range(15):
    for i in range(test_data_input.shape[0]):


        print('Processing {} th of {} images ... '.format(i, test_data_prediction.shape[0]))
        
        this_prediction = torch.zeros(iterate_count, 1, 64, 1024)
        for j in range(iterate_count):
            this_test = test_data_input[i]
            low_res = torch.from_numpy(this_test).to(device)
            low_res = torch.unsqueeze(low_res, dim=0)
            low_res = torch.moveaxis(low_res, 3, 2)
            low_res = torch.moveaxis(low_res, 2, 1)
            ooo = global_model(low_res.float(), S, 1, device, blocks)
            this_prediction[j] = ooo.cpu().detach()

        this_prediction = torch.moveaxis(this_prediction, 2, 1)
        this_prediction = torch.moveaxis(this_prediction, 3, 2)
        this_prediction = this_prediction.cpu().detach().numpy()
  
        this_prediction_mean = np.mean(this_prediction, axis=0)
        this_prediction_var = np.std(this_prediction, axis=0)
        test_data_prediction[i,:,:,0:1] = this_prediction_mean
        test_data_prediction[i,:,:,1:2] = this_prediction_var
        

    return test_data_prediction

In [13]:
global_model = torch.load("/home/alexgi/Workspace/lidar_super_resolution/Lidar Super-Resolution/Federated Learning/FL_DU_models/use_homo_encryption_6blocks_newTrue/global_model_round_20.pt")
predImages = MC_drop(iterate_count = 40)

# predImages = predImages *100

np.save('/home/alexgi/Workspace/lidar_super_resolution/Lidar Super-Resolution/homo_round_20.npy', predImages)

1
Processing 0 th of 8825 images ... 
Processing 1 th of 8825 images ... 
Processing 2 th of 8825 images ... 
Processing 3 th of 8825 images ... 
Processing 4 th of 8825 images ... 
Processing 5 th of 8825 images ... 
Processing 6 th of 8825 images ... 
Processing 7 th of 8825 images ... 
Processing 8 th of 8825 images ... 
Processing 9 th of 8825 images ... 
Processing 10 th of 8825 images ... 
Processing 11 th of 8825 images ... 
Processing 12 th of 8825 images ... 
Processing 13 th of 8825 images ... 
Processing 14 th of 8825 images ... 
Processing 15 th of 8825 images ... 
Processing 16 th of 8825 images ... 
Processing 17 th of 8825 images ... 
Processing 18 th of 8825 images ... 
Processing 19 th of 8825 images ... 
Processing 20 th of 8825 images ... 
Processing 21 th of 8825 images ... 
Processing 22 th of 8825 images ... 
Processing 23 th of 8825 images ... 
Processing 24 th of 8825 images ... 
Processing 25 th of 8825 images ... 
Processing 26 th of 8825 images ... 
Processin

In [ ]:
#Open3D: Visualize the point cloud
import open3d as o3d
import os, sys
import numpy as np


image_rows_low = 16 # 8, 16, 32
image_rows_high = 64 # 16, 32, 64
image_rows_full = 64
image_cols = 1024
# ouster
ang_res_x = 360.0/float(image_cols) # horizontal resolution
ang_res_y = 33.2/float(image_rows_high-1) # vertical resolution
ang_start_y = 16.6 # bottom beam angle
sensor_noise = 0.03
max_range = 85.0
min_range = 2.0

upscaling_factor = int(image_rows_high / image_rows_low)

rowList = []
colList = []
for i in range(image_rows_high):
    rowList = np.append(rowList, np.ones(image_cols)*i)
    colList = np.append(colList, np.arange(image_cols))

verticalAngle = np.float32(rowList * ang_res_y) - ang_start_y
horizonAngle = - np.float32(colList + 1 - (image_cols/2)) * ang_res_x + 90.0

verticalAngle = verticalAngle / 180.0 * np.pi
horizonAngle = horizonAngle / 180.0 * np.pi

intensity = rowList + colList / image_cols

In [ ]:
    # path_image = '/home/alexgi/Workspace/lidar_super_resolution/Lidar Super-Resolution/Federated Learning/FL_DU_models/use_homo_encryption_True/predImages_global_model_FL_homo_50_rounds.npy'
    path_image = '/home/alexgi/Workspace/lidar_super_resolution/Lidar Super-Resolution/Federated Learning/FL_DU_models/use_homo_encryption_6blocks_spectral_normFalse/predImages_global_model_6_blocks_FL_no_homo_50_rounds_spectral_norm.npy'
    test_data = np.load('/home/alexgi/Workspace/lidar_super_resolution/Lidar Super-Resolution/ouster_range_image.npy')
    
    predImages = np.load(path_image)*100

    # predImages = np.load('/home/alexgi/Workspace/lidar_super_resolution/Lidar Super-Resolution/My Predicted Lidar Images_DU_centralized/predImages_6blocks_dr005_modelspectral_ouster3_50.npy')*100
    # predImages = predImages.astype(np.float32, copy=True)

    upscaling_factor = int(image_rows_high / image_rows_low)
    
    
    low_res_index = range(0, image_rows_high, upscaling_factor)
    predImages[:,low_res_index,:,0:1] = np.copy(test_data[:,low_res_index,:,0:1]) # copy some of beams from origImages to predImages
    origImagesSmall = np.zeros(test_data.shape, dtype=np.float32) # for visualizing NN input images
    origImagesSmall[:,low_res_index] = test_data[:,low_res_index]

    predImagesNoiseReduced = np.copy(predImages[:,:,:,0:1])

    # remove noise
    if len(predImages.shape) == 4 and predImages.shape[-1] == 2:
        noiseLabels = predImages[:,:,:,1:2]
        predImagesNoiseReduced[noiseLabels > predImagesNoiseReduced * 0.05] = 0 # after noise removal
        predImagesNoiseReduced[:,low_res_index] = test_data[:,low_res_index]

    # predImages[:,:,:,1:2] = None


    thisImage = predImagesNoiseReduced[40,:,:,0]
    # thisImage = test_data[10,:,:,0]
    # thisImage = predImages[10,:,:,0]
    # thisImage = origImagesSmall[10,:,:,0]

    lengthList = thisImage.reshape(image_rows_high*image_cols)

    if len(thisImage.shape) == 3:
        print('check')
        thisImage = thisImage[:,:,0]

    lengthList = thisImage.reshape(image_rows_high*image_cols)
    lengthList[lengthList > max_range] = 0.0
    lengthList[lengthList < min_range] = 0.0

    x = np.sin(horizonAngle) * np.cos(verticalAngle) * lengthList
    y = np.cos(horizonAngle) * np.cos(verticalAngle) * lengthList
    z = np.sin(verticalAngle) * lengthList 

    points = np.column_stack((x,y,z))
    # delete points that has range value 0
    points = np.delete(points, np.where(lengthList==0), axis=0) # comment this line for visualize at the same speed (for video generation)


    # publish point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    # pcd.points["intensities"] = o3d.utility.Vector3dVector(intensity)
    o3d.visualization.draw_geometries([pcd])

In [ ]:
test_data_input, test_data = load_test_data()
from sklearn.metrics import mean_absolute_error

# predImages = np.load('/home/alexgi/Workspace/lidar_super_resolution/Lidar Super-Resolution/predImages_global_model_6_blocks_FL_no_homo_50_new.npy')
predImages = np.load('/home/alexgi/Workspace/lidar_super_resolution/Lidar Super-Resolution/Federated Learning/FL_DU_models/use_homo_encryption_6blocks_spectral_normFalse/predImages_global_model_6_blocks_FL_no_homo_50_rounds_spectral_norm.npy')

# full_res_data = test_data_prediction
this_test = np.empty([1, 16, 1024, 1], dtype=np.float32)
# gttest = np.empty([1, 64, 1024, 1], dtype=np.float32)

l1_error = []
for i in range(test_data.shape[0]):
        # print('Test image: ', i)
        # this_test[0,:,:,0] = test_data_input[i,:,:,0]
        
        # this_prediction = model.predict(this_test)
        pred = predImages[i,:,:,0]
        # pred[pred > max_range] = 0.0
        # pred[pred < min_range] = 0.0

        gt = test_data[i,:,:,0]
        # gt[gt > max_range] = 0.0
        # gt[gt < min_range] = 0.0
        
        l1_error.append(mean_absolute_error(pred, gt))

print('L1 error: ', np.mean(l1_error))

In [ ]:
#Open3D: publish_to_point_cloud
import open3d as o3d
import os, sys
import numpy as np


def publish_to_point_cloud(predImages, test_data):
    image_rows_low = 16 # 8, 16, 32
    image_rows_high = 64 # 16, 32, 64
    image_rows_full = 64
    image_cols = 1024
    # ouster
    ang_res_x = 360.0/float(image_cols) # horizontal resolution
    ang_res_y = 33.2/float(image_rows_high-1) # vertical resolution
    ang_start_y = 16.6 # bottom beam angle
    sensor_noise = 0.03
    max_range = 50.0
    min_range = 2.0

    upscaling_factor = int(image_rows_high / image_rows_low)

    rowList = []
    colList = []
    for i in range(image_rows_high):
        rowList = np.append(rowList, np.ones(image_cols)*i)
        colList = np.append(colList, np.arange(image_cols))

    verticalAngle = np.float32(rowList * ang_res_y) - ang_start_y
    horizonAngle = - np.float32(colList + 1 - (image_cols/2)) * ang_res_x + 90.0

    verticalAngle = verticalAngle / 180.0 * np.pi
    horizonAngle = horizonAngle / 180.0 * np.pi

    intensity = rowList + colList / image_cols
    
    #------------------------------------------------------------------

    upscaling_factor = int(image_rows_high / image_rows_low)
       
    low_res_index = range(0, image_rows_high, upscaling_factor)
    predImages[:,low_res_index,:,0:1] = np.copy(test_data[:,low_res_index,:,0:1]) # copy some of beams from origImages to predImages
    origImagesSmall = np.zeros(test_data.shape, dtype=np.float32) # for visualizing NN input images
    origImagesSmall[:,low_res_index] = test_data[:,low_res_index]

    predImagesNoiseReduced = np.copy(predImages[:,:,:,0:1])
    for ii in range(8200):

        thisImage = predImagesNoiseReduced[ii,:,:,0]
        lengthList = thisImage.reshape(image_rows_high*image_cols)

        if len(thisImage.shape) == 3:
            print('check')
            thisImage = thisImage[:,:,0]

        lengthList = thisImage.reshape(image_rows_high*image_cols)
        lengthList[lengthList > max_range] = 0.0
        lengthList[lengthList < min_range] = 0.0

        x = np.sin(horizonAngle) * np.cos(verticalAngle) * lengthList
        y = np.cos(horizonAngle) * np.cos(verticalAngle) * lengthList
        z = np.sin(verticalAngle) * lengthList 

        points = np.column_stack((x,y,z))
        # delete points that has range value 0
        points = np.delete(points, np.where(lengthList==0), axis=0) # comment this line for visualize at the same speed (for video generation)

        filename = f"/home/alexgi/Documents/DSP_point_clouds/Newtest_{ii}.ply"

        # publish point cloud
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        # pcd.points["intensities"] = o3d.utility.Vector3dVector(intensity)
        o3d.io.write_point_cloud(filename, pcd)
        # o3d.visualization.draw_geometries([pcd])    


In [ ]:
publish_to_point_cloud(predImages*100, test_data)